## Hello, Joel!

I just wanted to create a quick little demo of a very, _very_ basic web scraper I made for GFS data.  I have not worked with GFS data in the past, so I have had to make some assumptions on the kind of functionality you might be looking for.  But all webscrapers work roughly the same with varying degress of complexity.  The good news is that the NOAA website that hosts the GFS data is easily scraped.

And once I started looking at the GFS data, I can completely understand why there might be need of such a webscraper!  The `.grb2` file format does not have a lot of modern tools for dealing with it.  I haven't spent any time understanding the different raster bands or a lot about the file format.  Instead, I wanted to just provide ~1 hours worth of work to pull data in quickly and provide a quick image.  I am sure the file names make more sense to you who has worked with this data more, but here is what I have so far...

### What this script does

I am not sure exactly which product you want to use, so I have assumed GFS4.  From there, I have created something that will download the complete `.grb2` file based on date.  I note that the format of the files is `gfs_4_yyyymmdd_xxxx_yyy.grb2`.  The date is given by `yyyymmdd` and the time of measurement (in Zulu) is `xxxx`.  I am not entirely certain what `yyy` is, but I left it as a selectable parameter in the file downloading, noting that it increases by 3's from 0 to 384.  

So the user has to input the date that they are interested in as well as the collection time and "band" (the `yyy` above) and this script will download the appropriate `.grb2` file (which takes a couple of minutes based on file size) and create a quick image of whichever Raster Band is selected on the slider (I noted from some of the files I looked at that this appeared to go up to about 588).

### Some notes about this script

1. It is _far_ from optimized!  As it is, every time you click that button, the file is (re)downloaded and imaged. 
2. It is not at all what I would consider "production" code.  The packages that were used below were assembled rather hodge podge.  If I were to provide you with something that you actually wanted to use, we would have to set up your computer to hot have to worry about the sausage-making that went into making this code work on my computer.  This can easily be done through proper containerization, but in the roughly hour that I spent making this work I didn't spend time doing that.
3. The data isn't on any map.  There are several ways to do this in Python, but the documented ones with `.grb2` files are all deprecated and require a great deal of life support at present.  Suffice it to say, you are looking at the complete globe and I know you would rather focus on targetted regions.  That could be done with a bit more coding.
4. This really is not interactive.  While there are dropdowns that you can use, you cannot presently interact with the data in any meaningful fashion. However, knowing a bit more about what types of interactions you would be looking for would allow me to tailor an experience to that.

### Where this could go in the future

I am not going to claim that machine learning can predict the weather.  But there is a lot of data here and ML could even help just weed through the chaff a bit, bringing your attention to the meteorological features of interest or correlations to snowfall...or even just help identify outliers in the data here.  There is so much that could be done!

In [1]:
%matplotlib inline

import urllib.request
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact_manual

This bit here is the meat of the code that handles all interactions.  Note that I have limited the "band" entry to just a few values, but know that there are many more.  This is just for demonstration purposes.  The code is not the important part.  What you want is the drop downs and sliders below it.

In [12]:
def get_grib(start_date, collect_time, band, raster_band):
    date_str = start_date.strftime("%Y%m%d")
    year_month = start_date.strftime("%Y%m")
    file = 'gfs_4_' + date_str + '_' + collect_time + '_' + band + '.grb2'
    site = 'https://nomads.ncdc.noaa.gov/data/gfs4/' + year_month + '/' + date_str + '/' + file
    print('Getting grib...')
    urllib.request.urlretrieve(site, file)
    print('Got grib!')
    grib = gdal.Open(file)
    band = grib.GetRasterBand(raster_band)
    data = band.ReadAsArray()
    plt.imshow(data, cmap='jet')
    #return site
    
interact_manual(get_grib,
        start_date = widgets.DatePicker(description = 'Date: '),
        collect_time = widgets.Dropdown(options = ['0000', '0600', '1200', '1800'], description='Time of data (Zulu): '),
        band = widgets.Dropdown(options = ['000', '003', '006', '009', '012', '015'], description='Band: '),
        raster_band = widgets.IntSlider(min=1, max=588, description='Raster Band: '))

interactive(children=(DatePicker(value=None, description='Date: '), Dropdown(description='Time of data (Zulu):…

<function __main__.get_grib(start_date, collect_time, band, raster_band)>